In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 8.5MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates In

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import sys
import pandas as pd

In [3]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [4]:
review_dict = {'id_review':[], 'user':[], 'review':[], 'ratings':[], 'pub_date':[]}

In [5]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
url = "https://www.google.tn/maps/place/Tunisia+Mall/@36.846847,10.275538,15z/data=!4m7!3m6!1s0x12fd4ab44df809c9:0x97b81064f47410e2!8m2!3d36.8479914!4d10.2788652!9m1!1b1?hl=fr"
driver.get(url)
wait = WebDriverWait(driver, 10)
sleep(5)

for i in range(15):
    scrollable_div = driver.find_element_by_css_selector( 'div.section-layout.section-scrollbox.scrollable-y.scrollable-show')
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
    wait = WebDriverWait(driver, 10)
    sleep(5)

response = BeautifulSoup(driver.page_source, 'html.parser')
rlist = response.find_all('div', class_='section-review-content')

for reviews in rlist:
    
    id_r = reviews.find('button', class_='section-review-action-menu')['data-review-id']
    review_dict['id_review'].append(id_r)

    username = reviews.find('div', class_='section-review-title').find('span').text
    review_dict['user'].append(username)

    try:
        reviews.find('span', class_='section-review-text').text
        review_dict['review'].append(reviews.find('span', class_='section-review-text').text)
            
    except Exception:
        review_text = None
        review_dict['review'].append(review_text)
        
    rating = reviews.find('span', class_='section-review-stars')['aria-label']
    review_dict['ratings'].append(rating)
    
    rel_date = reviews.find('span', class_='section-review-publish-date').text
    review_dict['pub_date'].append(rel_date)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [6]:
df = pd.DataFrame.from_dict(review_dict, orient='index').transpose()

In [7]:
df=df.drop(['id_review','user','pub_date'], axis = 1)

In [8]:
df.to_csv('google_user_reviews.csv')